In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
XYDataFileName = "XY_n1000_p100_rep20.txt";
dataFolderPath = "data/simData";
'''
Run MGM
Note: MGM was implemented in Java and the following Python APIs call the Java implementation.
Please restart the Python program after encountering a JVM problem.
The format of the input data file must be ".txt" in which columns are separated by "\t" and it should also include the response variables.
Here is what the input data should look like:
X1 X2 ... Xp
1  1  ... 1
'''
# import the MGM package
from MGM.MGM import MGM
# Initialize a MGM object
mgm = MGM();
'''
Run MGM
Parameters:
    dataFolderPath: the directory that stores the input data.
    DataFileName: the name of the input data.
    lambda_continuous_continuous: the panalty value 'lambda' set for the associations whose two end variables are continuous.
    lamda_continuous_discrete: the panalty value 'lambda' set for the associations whose one end variable is continuous and the other is discrete.
    lamda_discrete_discrete: the panalty value 'lambda' set for the associations whose two end variables are discrete.
    
Return:
    mgmOutputFile: a tuple, where the first file contains all the selected associations and the second file contains the corresponding likelihoods.
'''
mgmOutputFile = mgm.runMGM(dataFolderPath, XYDataFileName,lambda_continuous_continuous = 0.3, lamda_continuous_discrete = 0.3, lamda_discrete_discrete = 0.3);
"""
MGM uses the Python package Jpype to call MGM's Java implementation.
According to Jpype documents, it says "Due to limitations in the JPype, 
it is not possible to restart the JVM after being terminated."
Therefore, please restart the Python kernel if you encounter an OSError (i.e., "OSError: JVM cannot be restarted").
"""
print("MGM's output was saved as the following file:");
print(mgmOutputFile[0]);
print("The likelihood values were saved as the following file:");
print(mgmOutputFile[1]);

/ihome/hpark/zhf16/causalDeepVASE/MGM/tetradLite_likelihood_for_all.jar
MGM's output was saved as the following file:
/ihome/hpark/zhf16/causalDeepVASE/data/simdata/XY_n1000_p100_rep20_MGM_associations.csv
The likelihood values were saved as the following file:
/ihome/hpark/zhf16/causalDeepVASE/data/simdata/XY_n1000_p100_rep20_likelihood_vals.txt


In [6]:
'''
#Generate knockoff data using one of two methods: ISEE Omega and Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

DataFileName = "X_n1000_p100_rep20.txt";
knockoffFilePath = generator.CholLuKnockoff(dataFolderPath, DataFileName,sep="\t");

# #If want to generate ISEE Omega knockoff, please set the ISEE code path and R home environment.
# generator.set_ISEE_path("/absolute_path_of_DAG_DeepVASE");#/home/user/DAG_DeepVASE/
# generator.set_R_home('absolute_path_to_directory_where_r_is_installed');#e.g.,/home/user/lib/R
# knockoffFilePath = generator.ISEEKnockoff(dataFolderPath, DataFileName,sep="\t");

print("The newly generated knockoff file is named as:");
print(knockoffFilePath);

KnockoffGenerator__init__


R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following objects are masked from ‘package:stats’:

    decompose, spectrum


R[write to console]: The following object is masked from ‘package:base’:

    union


R[write to console]: 
Attaching package: ‘pracma’


R[write to console]: The following objects are masked from ‘package:Matrix’:

    expm, lu, tril, triu




[1] 1000
[1] 100
/ihome/hpark/zhf16/causalDeepVASE/data/simdata/X_n1000_p100_rep20_knockoff.txt
The newly generated knockoff file is named as:
/ihome/hpark/zhf16/causalDeepVASE/data/simdata/X_n1000_p100_rep20_Omega_knockoff.txt


In [7]:
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t");
XKnockoffData

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,K91,K92,K93,K94,K95,K96,K97,K98,K99,K100
0,-1.201524,0.898138,-0.003530,-2.205407,4.598482,-4.619749,2.905533,-2.573103,2.623427,-4.371189,...,2.087498,2.454854,1.635319,1.114497,2.755592,-0.785129,3.010930,-5.333543,2.401977,-1.835576
1,-0.496584,3.059021,-4.155960,2.541485,-2.135378,2.039408,-3.655907,3.601994,-3.030936,4.128986,...,-4.880536,-0.172603,-3.218021,-0.559082,-3.650996,3.634668,-5.081293,7.071249,-3.740775,0.261924
2,1.252614,-1.112036,1.828080,-4.186963,3.564742,-5.534020,2.813050,-3.858096,5.373190,-5.290898,...,3.615820,-0.446997,2.662374,1.300380,0.801620,3.079058,-2.870500,0.200729,-0.638666,0.343097
3,2.450330,-2.595322,2.441193,-3.124635,2.184029,-1.806561,4.621149,-6.768368,5.826684,-4.227759,...,-7.939501,6.570805,-6.229781,5.942338,-5.891172,5.660287,-4.183954,1.390903,-4.496184,2.500434
4,1.289101,0.997923,0.560064,-0.807390,2.755672,-3.311042,2.264013,-1.858544,1.371533,-1.290510,...,1.870349,0.397886,2.479871,1.015436,1.689015,-4.252110,3.672500,-7.154860,3.437255,1.230302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.059347,-1.418195,2.368711,-1.026854,1.362450,-2.203226,1.331924,-1.139655,1.015572,1.898210,...,1.732673,-1.689766,3.563640,-2.488098,0.525998,-0.487005,-2.005315,0.251461,-0.578289,0.612767
996,1.832936,-1.671940,0.681624,0.522791,0.825174,-3.176672,3.446506,-3.902425,2.189422,-1.529924,...,-3.157309,0.380659,-3.394855,2.558910,-4.244344,2.599873,-0.355533,2.210841,-2.601619,2.097460
997,-0.456432,0.660194,-1.507720,3.498487,-3.938588,2.416287,-3.385438,2.691997,-1.151939,1.469998,...,3.860321,-1.474547,5.339228,-1.739301,4.961468,-3.245680,2.922389,-3.765520,2.845026,-1.415043
998,3.720271,-1.795586,0.832676,-3.502559,3.567162,-4.503909,2.554485,-2.703428,4.651822,-4.095047,...,-6.587422,4.313826,-4.951712,2.649282,-2.435342,2.268047,0.201895,-2.317514,-0.619590,-0.575980


In [8]:
''''''
# After generating the knockoff data, run DNN
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t");

YDataFileName = 'y_si_n1000_p100_rep20.txt';
Ydata = pd.read_csv(dataFolderPath+os.path.sep+YDataFileName,sep="\t");

XKValues = XKnockoffData.values;
YValues = Ydata.values;
    
pNum = int(XKValues.shape[1] / 2);
n = XKValues.shape[0];
print(XKValues.shape);
print(YValues.shape);
print(pNum);
    
XOrigin = XKValues[:, 0:pNum];
knockoff = XKValues[:, pNum:];

X3DTrain = np.zeros((n, pNum, 2));
X3DTrain[:, :, 0] = XOrigin;
X3DTrain[:, :, 1] = knockoff;
labelTrain = YValues;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(pNum) / n);

nOutputs = Ydata.shape[1];

#Save the DNN output to the following directory.
resultDir = dataFolderPath+os.path.sep+'DNN_result/';
if not os.path.exists(resultDir):
    os.makedirs(resultDir);
    
from DL.DNN.DNN import DNN;
dnn = DNN();
model = dnn.build_DNN(pNum, nOutputs, coeff);
callback = DNN.Job_finish_Callback(resultDir,pNum);
dnn.train_DNN(model, X3DTrain, labelTrain,callback);

(1000, 200)
(1000, 1)
100


Using TensorFlow backend.


__init__parameters
[layer]: Input	[shape]: [None, 100, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 100, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 100, 1] 

[layer]: Flatten	[shape]: [None, None] 

[layer]: Dense	[shape]: [None, 100] 

[layer]: Dense	[shape]: [None, 100] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/20
1000/1000 [==============================] - 4s 4ms/step - loss: 29442282.9084
on_epoch_end
h_local1_weight = (100, 2, 1)
h_local2_weight = (100, 1, 1)
h0 = (100, 2)
h0_abs = (100, 2)
h1 = (100, 100)
h2 = (100, 100)
h3 = (100, 1)
W1 = (100, 100)
W2 = (100, 100)
W3 = (100, 1)
Epoch 2/20
1000/1000 [==============================] - 0s 197us/step - loss: 29439798.1000
on_epoch_end
h_local1_weight = (100, 2, 1)
h_local2_weight = (100, 1, 1)
h0 = (100, 2)
h0_abs = (100, 2)
h1 = (100, 100)
h2 = (100, 100)
h3 = (100, 1)
W1 = (100, 100)
W2 = (100, 100)
W3 = (100, 1)
Epoch 3/20
1000/1000 [==============================] - 0s 181us/step - loss: 29406025.1681
on_epoch_en

In [13]:
#Apply FDR control to DNN result
from DL.FDR.FDR_control import FDR_control;
control = FDR_control();
XDataFileName = "X_n1000_p100_rep20.txt";
selected_features = control.controlFilter(dataFolderPath +os.path.sep+ XDataFileName, resultDir, offset=1, q=0.05);
#Save the selected associations
selected_associations = [];
for ele in selected_features:
    selected_associations.append({"Feature1":ele[0],"Feature2":"Y"});
pd.DataFrame(selected_associations).to_csv(dataFolderPath +os.path.sep+"DNN_selected_associations.csv")

__init__
200


In [21]:
# #Run DG
# #Load data
dataset = pd.read_csv(dataFolderPath+os.path.sep+XYDataFileName,sep="\t");

#Calculate the covariance matrix
cov_mat = dataset.cov();
corr_inv = np.linalg.inv(cov_mat)
corr_inv = pd.DataFrame(data=corr_inv, index=cov_mat.index,columns=cov_mat.columns)

#Convert the columns to their numerical representations
col_map = {};
col_map_rev = {};
col_list = dataset.columns.to_list();
for index,ele in enumerate(col_list):
    col_map[ele] = index;
    col_map_rev[index] = ele;
print(dataset.shape);

#The data may need to be normalized if neccessary.
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler();
# scaled_values = scaler.fit_transform(dataset);
# dataset.loc[:,:] = scaled_values;

#Initialize DG object
from causal.DegenerateGaussianScore import DegenerateGaussianScore
dg = DegenerateGaussianScore(dataset,discrete_threshold=0.2);

(1000, 101)


In [22]:
selectedAssociationsSum = [];
#Load both MGM-identified and DNN associations
MGMAssociations = pd.read_csv(mgmOutputFile[0]);
for index,row in MGMAssociations.iterrows():
    f1 = row["Feature1"];
    f2 = row["Feature2"];
    if f1=="Y" or f2=="Y":
        tempList = [f1,f2];
        tempList.sort();
        tempStr = tempList[0]+"___"+tempList[1];
        if tempStr not in selectedAssociationsSum:
            selectedAssociationsSum.append(tempStr);
        
DNNAssociations = pd.read_csv(dataFolderPath +os.path.sep+"DNN_selected_associations.csv");
for index,row in DNNAssociations.iterrows():
    f1 = row["Feature1"];
    f2 = row["Feature2"];
    tempList = [f1,f2];
    tempList.sort();
    tempStr = tempList[0]+"___"+tempList[1];
    if tempStr not in selectedAssociationsSum:
        selectedAssociationsSum.append(tempStr);

In [25]:
import networkx as nx
#Calculate causal directions
causalGraph = nx.DiGraph();
for ele in selectedAssociationsSum:
    strs = ele.split("___");
    f1 = strs[0];
    f2 = strs[1];
    
    inv_val = abs(corr_inv[f1][f2]);
    if inv_val<=0.0:
        continue;

    n1_idx = col_map[f1];
    n2_idx = col_map[f2];

    s1 = dg.localScore(n1_idx,{n2_idx});
    s2 = dg.localScore(n2_idx,{n1_idx});

    if s1<s2:
        print("Cause: "+f2+", Effect: "+f1);
        dif = s2-s1;
        causalGraph.add_edge(f2, f1, weight=dif);
    elif s1>s2:
        print("Cause: "+f1+", Effect: "+f2);
        dif = s1-s2;
        causalGraph.add_edge(f1, f2, weight=dif);
    else:
        print("Same score.");
#Remove cycles
causalGraph = dg.removeCycles(causalGraph);

Cause: F1, Effect: Y
Cause: F2, Effect: Y
Cause: F3, Effect: Y
Cause: F4, Effect: Y
Cause: F5, Effect: Y
Cause: F8, Effect: Y
Cause: F9, Effect: Y
Cause: F10, Effect: Y
Cause: F11, Effect: Y
Cause: F12, Effect: Y
Cause: F13, Effect: Y
Cause: F14, Effect: Y
Cause: F15, Effect: Y
Cause: F16, Effect: Y
Cause: F17, Effect: Y
Cause: F18, Effect: Y
Cause: F19, Effect: Y
Cause: F20, Effect: Y
Cause: F21, Effect: Y
Cause: F22, Effect: Y
Cause: F24, Effect: Y
Cause: F26, Effect: Y
Cause: F27, Effect: Y
Cause: F28, Effect: Y
Cause: F29, Effect: Y
Cause: F31, Effect: Y
Cause: F33, Effect: Y
Cause: F35, Effect: Y
Cause: F48, Effect: Y
Cause: F49, Effect: Y
Cause: F50, Effect: Y
Cause: F51, Effect: Y
Cause: F53, Effect: Y
Cause: F54, Effect: Y
Cause: F61, Effect: Y
Cause: F63, Effect: Y
Cause: F64, Effect: Y
Cause: F65, Effect: Y
Cause: F66, Effect: Y
Cause: F68, Effect: Y
Cause: F69, Effect: Y
Cause: F71, Effect: Y
Cause: F79, Effect: Y
Cause: F81, Effect: Y
Cause: F82, Effect: Y
Cause: F83, Effec

In [26]:
import scipy.stats
#Identify if a causal relationship is positive or negative and then save them.
edgeList = [];
for edge in causalGraph.edges():
    cause = edge[0];
    effect = edge[1];
    effectSize = np.log(causalGraph.get_edge_data(cause,effect)['weight']);
    corr = scipy.stats.pearsonr(dataset[cause].values,dataset[effect].values)[0];
    if corr>0:
        edgeList.append({"Cause":cause,"Effect":effect,"EffectSize":effectSize,"CauseDirection":"Positive"});
    elif corr == 0:
        edgeList.append({"Cause":cause,"Effect":effect,"EffectSize":effectSize,"CauseDirection":"Undefined"});
    else:
        edgeList.append({"Cause":cause,"Effect":effect,"EffectSize":effectSize,"CauseDirection":"Negative"});
    #print(corr);

In [27]:
pd.DataFrame(edgeList)

,Cause,Effect,EffectSize,CauseDirection
0,F1,Y,9.711081,Positive
1,F2,Y,9.675829,Negative
2,F3,Y,9.650835,Positive
3,F4,Y,9.630706,Negative
4,F5,Y,9.616693,Positive
5,F8,Y,9.583786,Negative
6,F9,Y,9.576426,Negative
7,F10,Y,9.568864,Positive
8,F11,Y,9.562412,Negative
9,F12,Y,9.554600,Positive
